In [121]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import string

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')

alice_txt = open('alice.txt', encoding='utf8').read()

'text downloaded'

[nltk_data] Downloading package wordnet to /home/patrik-
[nltk_data]     sh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/patrik-sh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/patrik-
[nltk_data]     sh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/patrik-
[nltk_data]     sh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'text downloaded'

In [124]:
#Text preprocessing

#remove all new lines, lowercase
alice_txt = alice_txt.replace(r'\n', ' ')
alice_txt = alice_txt.lower()

#delete the useless beginning and the ending 
#BEGINNING = '[Illustration]'
#ENDING = 'THE END'
#print(len(alice_txt.split(BEGINNING)))
#alice_txt = alice_txt.split(BEGINNING)[1]

alice_tokenized = pd.Series(word_tokenize(alice_txt))

alice_no_stopwords = alice_tokenized.loc[~alice_tokenized.isin(stopwords.words('english'))]

alice_no_punction = alice_no_stopwords.loc[~alice_no_stopwords.isin(list(string.punctuation + string.digits + "'" + '"' + '”' + '“' + '’'))]

lemmatizer = nltk.WordNetLemmatizer()
alice_preprocessed = alice_no_punction.apply(lambda word: lemmatizer.lemmatize(word))

alice_preprocessed

0             alice
3         adventure
5        wonderland
7             lewis
8           carroll
            ...    
34622    child-life
34626         happy
34627        summer
34628           day
34631           end
Length: 12507, dtype: object

In [128]:
#divide by chapters creating an dataframe structure [chapter,word]

chapterized_alice_arr = []

chapter_nmb = 0
for word in alice_preprocessed.iteritems():
  if (word[1] == 'chapter'):
    if chapter_nmb == 12:
      chapter_nmb = 1
    else:
      chapter_nmb += 1
  chapterized_alice_arr.append({ 'chapter': chapter_nmb, 'word': word[1] })

chapterized_alice = pd.DataFrame(chapterized_alice_arr)

def get_10_frequent_words(df: pd.DataFrame, chapter: int):
  df_working = df[df.chapter == chapter]
  unique_words = df_working[df_working.word != 'alice'].word.unique()

  words_count = []

  for w in unique_words:
    words_count.append({
      'word': w,
      'count': df_working.loc[df_working.word == w].word.count()
    })
  
  counted = pd.DataFrame(words_count)
  return counted.sort_values(by=['count'], ascending=False).head(10)

for chap in range(1,13):
  print(f'----for chapter {chap}----')
  print(get_10_frequent_words(chapterized_alice, chap))


----for chapter 1----
       word  count
225  little     15
93     like     11
50      way     11
84      see     10
48    think      9
296    door      9
126     one      8
61     time      8
22    could      8
159    said      8
----for chapter 2----
       word  count
367   mouse     16
29   little     16
107    said     12
34     dear     11
52       go     11
20     foot     10
171   thing     10
15     like      9
2      pool      9
42     must      9
----for chapter 3----
      word  count
55    said     34
56   mouse     21
170   dodo     12
45    know     11
90       ‘      9
202    one      8
3     long      7
65    soon      7
10    bird      6
41   would      6
----for chapter 4----
        word  count
4     little     24
2     rabbit     16
199      one     14
77      said     14
5       bill     13
225  thought      9
25      sure      9
15     heard      9
23       get      9
113    thing      8
----for chapter 5----
            word  count
16          said     52
3    c